In [3]:
import os

In [4]:
def download_images(images_directory):
    all_images_paths = []
    images_directories = os.listdir(images_directory)
    for directory in images_directories:
        directory_relaitve_path = f"{images_directory}/{directory}"
        files = os.listdir(directory_relaitve_path)
        for file in files:
            if file.endswith(".png"):
                all_images_paths.append(f"{images_directory}/{directory}/{file}")
            else:
                 pass
    return all_images_paths

In [5]:
img_paths = download_images("/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb_Git_Repo/AirbnbDataSci/images")

In [13]:
import cv2

In [17]:
cv2.imread(img_paths[0]).shape

(618, 720, 3)